# Домашнее задание 1. Извлечение ключевых слов

### 1. Подготовить мини-корпус (не меньше 4 текстов, примерный общий объём - 3-5 тысяч токенов) с разметкой ключевых слов. Предполагается, что вы найдете источник текстов, в котором уже выделены ключевые слова. Укажите источник корпуса и опишите, в каком виде там были представлены ключевые слова.

Я взяла тексты с сайта CNN. У них в HTML коде страницы для многих новостей вшиты ключевые слова под тегом `meta name="keywords
content="..."`, например, 
```
(угловая скобка) meta name="keywords" content="arts and entertainment, basketball, celebrity and pop culture, companies, internet and www, luka doncic, nba, social media, sports and recreation, sports figures, sports organizations and teams, technology, twitter" (угловая скобка)

```

Так как эти ключевые слова есть не для всех текстов, а сами тексты могут быть разной длины (это может быть и opinion piece длиной 3000 слов), то я отбирала тексты вручную.

In [43]:
corpus = open('cnn_corpus.txt', encoding='UTF-8').read()
    
corpus = corpus.split('\n—\n')
print(corpus[1]) #пример текста

NBA SUPERSTAR LUKA DONCIC SAYS HE PREFERS CHESS TO TWITTER
Much of the world seems to be talking about Elon Musk’s takeover of Twitter and his decision to charge $8 a month for a blue tick, but Luka Doncic appears to have more cerebral pursuits on his mind when he switches on his phone.
After Musk’s $44 billion acquisition of Twitter was explained to the 23-year-old NBA star by a reporter at a media conference on Tuesday, all Doncic could respond with was a shrug of his shoulders and a simple reply.
“I just play chess on my phone,” said the Dallas Mavericks star.
The three-time All-NBA first teamer’s response prompted a follow-up question asking if the computer usually wins or if the basketball player does when he plays chess.
His answer was so typically Doncic: “I play online. But I win, mostly.”
It’s not the first time he’s shown his love for chess.
After his game-winning performance against the Boston Celtics in February 2021, Doncic was asked how he chooses how to dissect a defense

In [44]:
print("Total texts", len(corpus))
print("Total words", sum(len(i.split()) for i in corpus))
print("Text lengths", list(len(i.split()) for i in corpus))

Total texts 10
Total words 5153
Text lengths [246, 393, 751, 395, 338, 405, 398, 258, 790, 1179]


### 2. Разметить ключевые слова самостоятельно. Оценить пересечение с имеющейся разметкой. Составить эталон разметки (например, пересечение или объединение вашей разметки и исходной).

Так выглядят ключевые слова: сначала -- оригинальные CNN-овские, потом -- мои ручные

In [45]:
a = open('cnn_own_keywords.txt').read().split('\n\n') 
m = open('cnn_manual_keywords.txt').read().split('\n\n')
assert len(a) == len(m) == 10
for i in range(len(corpus)):
    print(a[i], '\n\n', m[i], '\n---')

asia, continents and regions, india, south asia, aviation and aerospace industry, business and industry sectors, business, economy and trade, space and astronomy, space industry, spacecraft and satellites, space exploration, space launches 

 india, rocket, skyroot, launch, vikram 
---
arts and entertainment, basketball, celebrity and pop culture, companies, internet and www, luka doncic, nba, social media, sports and recreation, sports figures, sports organizations and teams, technology, twitter 

 nba, doncic, musk, twitter, chess, blue, verified 
---
business and industry sectors, business, economy and trade, government and public administration, government organizations - us, labor and employment, labor disputes and negotiations, labor relations, labor strikes, labor unions, politics, rail transportation, railway, transportation and warehousing, transportation infrastructure, us congress, companies, csx corp, freight railway, freight transportation, compensation and benefits, emplo

Понятно, что у нас разные стратегии разметки ключевых слов -- во-первых, у них в начале идут общие темы и регионы, которых касается статья, даже если их названий в самой статье нет, во-вторых, у них многие из этих тем пересекаются (soccer events, sports events, sports and recreation как теги одной статьи и др.). 

In [46]:
for i in range(len(corpus)):
    print(set((j.strip() for j in a[i].split(','))).intersection(set((j.strip() for j in m[i].split(',')))))

{'india'}
{'twitter', 'nba'}
set()
set()
set()
{'mental health'}
{'australian open', 'tennis', 'australia'}
{'nigeria', 'africa'}
{'united kingdom', 'recession'}
set()


Пересечение очень маленькое, но надо заметить, что есть эквивалентные теги, записанные по-разному: ticketmaster и ticketmaster entertainment inc, manchester united fc и manchester united, doncic и luka doncic -- попробуем добавлять все совместные подстроки в ключевые слова.

In [47]:
gold_keywords = []
for i in range(len(corpus)):
    own = {j.strip() for j in a[i].split(',')}
    mine = {j.strip() for j in m[i].split(',')}
    common = set()
    for tag in own:
        if tag in m[i]:
            common.add(tag)
    for tag in mine:
        if tag in a[i]:
            common.add(tag)
            
    print(common)
    
    gold_keywords += [list(common)]

{'india', 'launch'}
{'twitter', 'doncic', 'nba'}
{'labor', 'strike', 'freight', 'union', 'industry'}
{'agriculture', 'food'}
{'ticketmaster', 'swift'}
{'mental health'}
{'australian open', 'grand slam', 'australia', 'djokovic', 'tennis'}
{'nigeria', 'africa', 'oil', 'economy', 'port'}
{'hunt', 'tax', 'recession', 'united kingdom', 'economy', 'government'}
{'ronaldo', 'team', 'champions league', 'united', 'manchester', 'season', 'manchester united'}


Наконец, есть случаи с названиями организаций -- у меня "fda", у них "us food and drug administration", но так как в тексте она называется именно FDA, то оставим там. Кроме того, я добавлю названия мест, организаций, событий, о которых идёт речь в статьях, которых нет в тегах CNN (как в статье с открытием нового порта в Нигерии надо добавить название самого порта -- Lekki -- так как оно часто встречается). Видимо, у них это не включается из-за слишком узкого значения.

In [48]:
to_add = [
    ['vikram', 'skyroot'], 
    [], 
    [], 
    ['fda', 'upside foods'], 
    ['eras tour'], 
    [], 
    [], 
    ['lekki', 'lagos'], 
    ['jefferies'], 
    ['ten hag']]

for i in range(len(gold_keywords)):
    gold_keywords[i] += to_add[i]
    
gold_keywords = list(set(i) for i in gold_keywords)
gold_keywords

[{'india', 'launch', 'skyroot', 'vikram'},
 {'doncic', 'nba', 'twitter'},
 {'freight', 'industry', 'labor', 'strike', 'union'},
 {'agriculture', 'fda', 'food', 'upside foods'},
 {'eras tour', 'swift', 'ticketmaster'},
 {'mental health'},
 {'australia', 'australian open', 'djokovic', 'grand slam', 'tennis'},
 {'africa', 'economy', 'lagos', 'lekki', 'nigeria', 'oil', 'port'},
 {'economy',
  'government',
  'hunt',
  'jefferies',
  'recession',
  'tax',
  'united kingdom'},
 {'champions league',
  'manchester',
  'manchester united',
  'ronaldo',
  'season',
  'team',
  'ten hag',
  'united'}]

### 3. Применить к этому корпусу 3 метода извлечения ключевых слов на выбор 

Сначала применим предобработку к тексту: уберём стоп-слова и лемматизируем.

In [49]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

stopwords = nlp.Defaults.stop_words

C:\Users\Katya\AppData\Roaming\Python\Python38\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.3.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [50]:
normalized_texts = [' '.join(list([token.lemma_.lower() for token in nlp(i) if token.lower_ not in stopwords])) for i in corpus]

In [51]:
print(normalized_texts[0])

india private rocket vikram - s launched space 
 indian startup skyroot aerospace launch country privately develop rocket , vikram - s , space friday support indian space research organisation ( isro ) . 
 , country space industry dominate state - run isro , skyroot aerospace open sector private company . 
 " rocket launch help test validate majority technology vikram series orbital class space launch vehicle , include sub - system technology test pre - lift post - lift phase launch , " accord skyroot aerospace . 
 rocket , weigh 546 kilogram ( 1203 pound ) , launch sriharikota spaceport reach altitude 89.5 kilometer ( 55.6 mile ) . 
 mission symbolize india private rocket launch " potential new india , " say pawan kumar chandana , co - founder skyroot aerospace launch . 
 skyroot aerospace launch 2018 base southern tech hub hyderabad . found isro engineer raise $ 68 million funding , accord datum firm tracxn . 
 year , startup enter agreement isro use indian space agency expertise acc

#### 1. TF-IDF

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer 
tfidf = TfidfVectorizer(ngram_range=(1,2))

In [53]:
tfidf_data = tfidf.fit_transform(normalized_texts)

In [84]:
import pandas as pd
feat = tfidf.get_feature_names_out()
tfidf_kw = []
for i in range(len(corpus)):
    doc = tfidf_data[i]
    df = pd.DataFrame(doc.T.todense(), index=feat, columns=['weights'])
    df = df.sort_values(by=['weights'], ascending=False)[:8] # в самом длинном списке ключевых слов 8 слов
    tfidf_kw += [df.index.tolist()]

In [55]:
tfidf_tp, tfidf_fn, tfidf_fp = [], [], []
for i in range(len(corpus)):
    print(f'TEXT {i+1}')
    tfidf_tp_i = set(tfidf_kw[i]).intersection(gold_keywords[i])
    tfidf_tp.append(tfidf_tp_i)
    tfidf_fn_i = gold_keywords[i].difference(set(tfidf_kw[i]))
    tfidf_fn.append(tfidf_fn_i)
    tfidf_fp_i = set(tfidf_kw[i]).difference(gold_keywords[i])
    tfidf_fp.append(tfidf_fp_i)
    print('keywords in common', tfidf_tp_i)
    print('keywords not found', tfidf_fn_i)
    print('extra keywords', tfidf_fp_i)

TEXT 1
keywords in common {'skyroot', 'launch'}
keywords not found {'vikram', 'india'}
extra keywords {'skyroot aerospace', 'rocket', 'isro', 'space', 'aerospace', 'private'}
TEXT 2
keywords in common {'twitter', 'doncic', 'nba'}
keywords not found set()
extra keywords {'chess', 'star', 'musk', 'account', 'play'}
TEXT 3
keywords in common {'strike', 'union'}
keywords not found {'freight', 'labor', 'industry'}
extra keywords {'railroad', 'agreement', 'sick', 'member', 'reach', 'reject'}
TEXT 4
keywords in common {'food', 'fda'}
keywords not found {'agriculture', 'upside foods'}
extra keywords {'foods', 'cultivate', 'upside', 'meat', 'chicken', 'animal'}
TEXT 5
keywords in common {'ticketmaster', 'swift'}
keywords not found {'eras tour'}
extra keywords {'fan', 'tour', 'ticket', 'buy', 'ticketmaster say', 'demand'}
TEXT 6
keywords in common {'mental health'}
keywords not found set()
extra keywords {'employee', 'health', 'issue', 'kuhn', 'employer', 'mental', 'employee mental'}
TEXT 7
keyw

#### 2. RAKE

In [57]:
import RAKE
rake = RAKE.Rake(list(stopwords))

In [58]:
rake_kw_ = []
for i in range(len(corpus)):
    rake_kw_ += [rake.run(normalized_texts[i], maxWords = 2, minFrequency=2)]

In [59]:
rake_kw = []
for rkw in rake_kw_:
    if rkw:
        rake_kw += [[i[0] for i in rkw]]
    else:
        rake_kw += [[]]

In [60]:
rake_kw

[['year'],
 ['year', 'time'],
 ['jefferies', 'union'],
 ['lab', 'fda'],
 ['ticketmaster'],
 [],
 [],
 [],
 ['difficult decision', 'hunt', 'selfin', '000'],
 ['club', 'respect', 'ronaldo', 'year', 'interview']]

In [61]:
rake_tp, rake_fn, rake_fp = [], [], []
for i in range(len(corpus)):
    print(f'TEXT {i+1}')
    rake_tp_i = set(rake_kw[i]).intersection(gold_keywords[i])
    rake_tp.append(rake_tp_i)
    rake_fn_i = gold_keywords[i].difference(set(rake_kw[i]))
    rake_fn.append(rake_fn_i)
    rake_fp_i = set(rake_kw[i]).difference(gold_keywords[i])
    rake_fp.append(rake_fp_i)
    print('keywords in common', rake_tp_i)
    print('keywords not found', rake_fn_i)
    print('extra keywords', rake_fp_i)

TEXT 1
keywords in common set()
keywords not found {'vikram', 'skyroot', 'india', 'launch'}
extra keywords {'year'}
TEXT 2
keywords in common set()
keywords not found {'twitter', 'doncic', 'nba'}
extra keywords {'year', 'time'}
TEXT 3
keywords in common {'union'}
keywords not found {'freight', 'strike', 'labor', 'industry'}
extra keywords {'jefferies'}
TEXT 4
keywords in common {'fda'}
keywords not found {'agriculture', 'food', 'upside foods'}
extra keywords {'lab'}
TEXT 5
keywords in common {'ticketmaster'}
keywords not found {'eras tour', 'swift'}
extra keywords set()
TEXT 6
keywords in common set()
keywords not found {'mental health'}
extra keywords set()
TEXT 7
keywords in common set()
keywords not found {'australian open', 'australia', 'djokovic', 'tennis', 'grand slam'}
extra keywords set()
TEXT 8
keywords in common set()
keywords not found {'oil', 'nigeria', 'economy', 'lagos', 'port', 'lekki', 'africa'}
extra keywords set()
TEXT 9
keywords in common {'hunt'}
keywords not found 

#### 3. TextRank

In [62]:
from summa import keywords

In [63]:
sum_kw_ = []
for i in range(len(corpus)):
    sum_kw_ += [keywords.keywords(normalized_texts[i])]

In [64]:
sum_kw = []
for i in sum_kw_:
    sum_kw += [i.split('\n')]

In [66]:
sum_tp, sum_fn, sum_fp = [], [], []
for i in range(len(corpus)):
    print(f'TEXT {i+1}')
    sum_tp_i = set(sum_kw[i]).intersection(gold_keywords[i])
    sum_tp.append(sum_tp_i)
    sum_fn_i = gold_keywords[i].difference(set(sum_kw[i]))
    sum_fn.append(sum_fn_i)
    sum_fp_i = set(sum_kw[i]).difference(gold_keywords[i])
    sum_fp.append(sum_fp_i)
    print('keywords in common', sum_tp_i)
    print('keywords not found', sum_fn_i)
    print('extra keywords', sum_fp_i)

TEXT 1
keywords in common set()
keywords not found {'vikram', 'skyroot', 'india', 'launch'}
extra keywords {'technology', 'india private rocket', 'isro', 'richard', 'launched space indian startup skyroot aerospace launch country privately', 'year', 'test', 'accord', 'lift', 'hub'}
TEXT 2
keywords in common {'nba'}
keywords not found {'twitter', 'doncic'}
extra keywords {'game', 'star', 'way', 'week', 'month blue', 'chess twitter', 'account', 'follow', 'phone', 'win basketball', 'play', 'performance', 'verify', 'old', 'court build', 'musk', 'gaming company', 'question ask', 'decision', 'year', 'doncic say'}
TEXT 3
keywords in common set()
keywords not found {'labor', 'strike', 'freight', 'union', 'industry'}
extra keywords {'worker', 'union member avoid', 'sick pay', 'action congress block', 'say', 'freight railroads strike avoided', 'day', 'contract', 'peb', 'literally', 'labor deal reach', 'provide', 'proposal', 'railroad', 'far', 'request', 'ratification vote', 'clearly', 'clear', 'r

### 4. Составить морфологические/синтаксические шаблоны для ключевых слов и фраз, выделить соответствующие им подстроки из корпуса (например, именные группы Adj+Noun). Применить эти фильтры к спискам ключевых слов.

In [67]:
gold_keywords

[{'india', 'launch', 'skyroot', 'vikram'},
 {'doncic', 'nba', 'twitter'},
 {'freight', 'industry', 'labor', 'strike', 'union'},
 {'agriculture', 'fda', 'food', 'upside foods'},
 {'eras tour', 'swift', 'ticketmaster'},
 {'mental health'},
 {'australia', 'australian open', 'djokovic', 'grand slam', 'tennis'},
 {'africa', 'economy', 'lagos', 'lekki', 'nigeria', 'oil', 'port'},
 {'economy',
  'government',
  'hunt',
  'jefferies',
  'recession',
  'tax',
  'united kingdom'},
 {'champions league',
  'manchester',
  'manchester united',
  'ronaldo',
  'season',
  'team',
  'ten hag',
  'united'}]

Поскольку у меня среди ключевых слов почти исключительно существительные, сложно придумать, какие маски можно применить. Попробуем формулы (Noun), (Adj + Noun) и (Noun + Noun).

In [68]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)

In [69]:
noun = [{'POS': {'IN': ['NOUN', 'PROPN']}}]
adj_noun = [{'POS':'ADJ'}, {'POS':'NOUN'}]
noun_noun = [{'POS':{'IN': ['NOUN', 'PROPN']}}, {'POS':{'IN': ['NOUN', 'PROPN']}}]
matcher.add('n', [noun])
matcher.add('an', [adj_noun], greedy='LONGEST')
matcher.add('nn', [noun_noun])

In [70]:
from tqdm import tqdm

In [71]:
matches = [[]]
for i in tqdm(range(len(corpus))):
    matches_ = matcher(nlp(corpus[i]))
    for m, s, e in matches_:
        matches[-1] += [nlp(corpus[i])[s:e]]
    matches += [[]]
    
del matches[-1]

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:32<00:00,  9.29s/it]


In [72]:
masked_texts = [' '.join([j.text for j in spacy.util.filter_spans(i)]) for i in matches]

In [73]:
masked_texts[3]

'LAB MEAT HUMAN CONSUMPTION FDA US Food Drug Administration safety clearance lab meat first time Upside Foods California company meat cultured chicken cells products facilities US Department Agriculture agency information Upside Foods further questions time firm safety conclusion lab meat planet health FDA goal innovation food technologies first priority safety foods U.S. consumers statement Upside Foods founder CEO Uma Valeti Twitter chicken step tables UPSIDE Questions Letter FDA Valeti conclusion chicken CNN year process meat brewing beer yeast microbes animal cells products vegan vegetarian plant real meat animal Singapore first country sale cultured meat San Francisco Inc. regulatory approval laboratory chicken Singapore Advocates cultured meat need animals food climate crisis food system quarter global greenhouse gas emissions animal agriculture FDA historic announcement rigorous evaluation UPSIDE Foods first company world US FDA greenlight chicken David Kay director communicatio

Посчитаем ключевые слова на новых данных:

In [74]:
# TD-IDF
tfidf_ = TfidfVectorizer(ngram_range=(1,2))
tfidf_m = tfidf_.fit_transform(masked_texts)

feat_m = tfidf_.get_feature_names_out()
tfidf_m_kw = []
for i in range(len(corpus)):
    doc = tfidf_m[i]
    df = pd.DataFrame(doc.T.todense(), index=feat_m, columns=['weights'])
    df = df.sort_values(by=['weights'], ascending=False)[:15]
    tfidf_m_kw += [df.index.tolist()]
    
tfidf_m_tp, tfidf_m_fn, tfidf_m_fp = [], [], []
for i in range(len(corpus)):
    print(f'TEXT {i+1}')
    tfidf_m_tp_i = set(tfidf_m_kw[i]).intersection(gold_keywords[i])
    tfidf_m_tp.append(tfidf_m_tp_i)
    tfidf_m_fn_i = gold_keywords[i].difference(set(tfidf_m_kw[i]))
    tfidf_m_fn.append(tfidf_m_fn_i)
    tfidf_m_fp_i = set(tfidf_m_kw[i]).difference(gold_keywords[i])
    tfidf_m_fp.append(tfidf_m_fp_i)
    print('keywords in common', tfidf_m_tp_i)
    print('keywords not found', tfidf_m_fn_i)
    print('extra keywords', tfidf_m_fp_i)

TEXT 1
keywords in common {'vikram', 'skyroot', 'india', 'launch'}
keywords not found set()
extra keywords {'skyroot aerospace', 'indian', 'rocket', 'startup', 'isro', 'space', 'indian space', 'aerospace', 'private', 'launch skyroot', 'private rocket'}
TEXT 2
keywords in common {'twitter', 'doncic', 'nba'}
keywords not found set()
extra keywords {'luka doncic', 'basketball', 'chess', 'star', 'mavericks', 'musk', 'account', 'decision', 'luka', 'blue', 'accounts', 'tick'}
TEXT 3
keywords in common {'labor', 'strike', 'freight', 'union', 'industry'}
keywords not found set()
extra keywords {'railroads', 'sick', 'congress', 'unions', 'agreements', 'sick days', 'members', 'workers', 'deals', 'jefferies'}
TEXT 4
keywords in common {'food', 'fda', 'upside foods'}
keywords not found {'agriculture'}
extra keywords {'lab', 'foods', 'upside', 'us', 'first', 'lab meat', 'meat', 'cultured', 'safety', 'cells', 'chicken', 'animal'}
TEXT 5
keywords in common {'eras tour', 'ticketmaster', 'swift'}
keywo

In [75]:
# RAKE
rake_m_kw_ = []
for i in range(len(corpus)):
    rake_m_kw_ += [rake.run(masked_texts[i], maxWords = 2, minFrequency=1)]
    
rake_m_kw = []
for rkw in rake_m_kw_:
    if rkw:
        rake_m_kw += [[i[0] for i in rkw]]
    else:
        rake_m_kw += [[]]


rake_m_tp, rake_m_fn, rake_m_fp = [], [], []
for i in range(len(corpus)):
    print(f'TEXT {i+1}')
    rake_m_tp_i = set(rake_m_kw[i]).intersection(gold_keywords[i])
    rake_m_tp.append(rake_m_tp_i)
    rake_m_fn_i = gold_keywords[i].difference(set(rake_m_kw[i]))
    rake_m_fn.append(rake_m_fn_i)
    rake_m_fp_i = set(rake_m_kw[i]).difference(gold_keywords[i])
    rake_m_fp.append(rake_m_fp_i)
    print('keywords in common', rake_m_tp_i)
    print('keywords not found', rake_m_fn_i)
    print('extra keywords', rake_m_fp_i)

TEXT 1
keywords in common {'india'}
keywords not found {'vikram', 'skyroot', 'launch'}
extra keywords {'year', 's'}
TEXT 2
keywords in common {'nba'}
keywords not found {'twitter', 'doncic'}
extra keywords set()
TEXT 3
keywords in common set()
keywords not found {'labor', 'strike', 'freight', 'union', 'industry'}
extra keywords {'freight railroads'}
TEXT 4
keywords in common set()
keywords not found {'agriculture', 'food', 'fda', 'upside foods'}
extra keywords {'s', 'company world'}
TEXT 5
keywords in common set()
keywords not found {'eras tour', 'ticketmaster', 'swift'}
extra keywords set()
TEXT 6
keywords in common set()
keywords not found {'mental health'}
extra keywords set()
TEXT 7
keywords in common set()
keywords not found {'australian open', 'grand slam', 'australia', 'djokovic', 'tennis'}
extra keywords {'world'}
TEXT 8
keywords in common {'nigeria'}
keywords not found {'africa', 'oil', 'economy', 'lagos', 'port', 'lekki'}
extra keywords {'construction'}
TEXT 9
keywords in com

In [76]:
# TextRank

sum_m_kw_ = []
for i in range(len(corpus)):
    sum_m_kw_ += [keywords.keywords(masked_texts[i])]
    
sum_m_kw = []
for i in sum_m_kw_:
    sum_m_kw += [i.split('\n')]

sum_m_tp, sum_m_fn, sum_m_fp = [], [], []
for i in range(len(corpus)):
    print(f'TEXT {i+1}')
    sum_m_tp_i = set(sum_m_kw[i]).intersection(gold_keywords[i])
    sum_m_tp.append(sum_m_tp_i)
    sum_m_fn_i = gold_keywords[i].difference(set(sum_m_kw[i]))
    sum_m_fn.append(sum_m_fn_i)
    sum_m_fp_i = set(sum_m_kw[i]).difference(gold_keywords[i])
    sum_m_fp.append(sum_m_fp_i)
    print('keywords in common', sum_m_tp_i)
    print('keywords not found', sum_m_fn_i)
    print('extra keywords', sum_m_fp_i)

TEXT 1
keywords in common {'skyroot', 'india', 'launch'}
keywords not found {'vikram'}
extra keywords {'indian', 'rocket', 'technologies', 'isro', 'space', 'aerospace country', 'private', 'launched', 'firm', 'jeff'}
TEXT 2
keywords in common {'twitter', 'doncic', 'nba'}
keywords not found set()
extra keywords {'chess', 'star', 'decision month blue', 'account', 'phone', 'time', 'question', 'basketball', 'court', 'game performance', 'questions', 'musk', 'gaming company', 'year', 'accounts'}
TEXT 3
keywords in common {'freight', 'strike', 'industry'}
keywords not found {'union', 'labor'}
extra keywords {'contracts railroad', 'deal agreement', 'nations', 'railroads', 'way', 'sick', 'union members', 'recommendation', 'peb', 'labor deals', 'recommendations member', 'action', 'employee', 'ratification', 'nation', 'congress', 'unions agreements', 'sides', 'workers', 'employees', 'best', 'jefferies', 'management', 'cnn'}
TEXT 4
keywords in common {'agriculture', 'food', 'fda'}
keywords not foun

### 5. Оценить точность, полноту, F-меру выбранных методов относительно эталона: с учётом морфосинтаксических шаблонов и без них.

Можно считать для каждого текста каждое значение по отдельности, а потом брать среднее для метода значение по всем текстам. А можно посчитать суммы всех текстов в корпусе (сколько всего слов угадал, сколько всего лишних и т.д.), и потом считать метрику по этим суммам. 

In [77]:
from statistics import mean

In [78]:
# mean TF-IDF across all texts

tfidf_precision, tfidf_recall, tfidf_f1 = [], [], []
for i in range(len(corpus)):
    tfidf_precision.append(len(tfidf_tp[i]) / (len(tfidf_tp[i]) + len(tfidf_fp[i])))
    tfidf_recall.append(len(tfidf_tp[i]) / (len(tfidf_tp[i]) + len(tfidf_fn[i])))
    tfidf_f1.append(2 * (tfidf_precision[i] * tfidf_recall[i]) / (tfidf_precision[i] + tfidf_recall[i]))
    
print(f'''
Mean TF-IDF precision: {mean(tfidf_precision)},
Mean TF-IDF recall: {mean(tfidf_recall)},
Mean TF-IDF F1-score: {mean(tfidf_f1)}
''')

# total TF-IDF as if all texts were one 

tfidf_precision = len([j for i in tfidf_tp for j in i]) / (len([j for i in tfidf_tp for j in i]) + len([j for i in tfidf_fp for j in i]))
tfidf_recall = len([j for i in tfidf_tp for j in i]) / (len([j for i in tfidf_tp for j in i]) + len([j for i in tfidf_fn for j in i]))
tfidf_f1 = 2 * (tfidf_precision * tfidf_recall) / (tfidf_precision + tfidf_recall)
print(f'''
Total TF-IDF precision: {tfidf_precision},
Total TF-IDF recall: {tfidf_recall},
Total TF-IDF F1-score: {tfidf_f1}
''')



Mean TF-IDF precision: 0.325,
Mean TF-IDF recall: 0.6166666666666667,
Mean TF-IDF F1-score: 0.40005439005439003


Total TF-IDF precision: 0.325,
Total TF-IDF recall: 0.5531914893617021,
Total TF-IDF F1-score: 0.4094488188976378



In [79]:
# mean RAKE across all texts

rake_precision, rake_recall, rake_f1 = [], [], []
for i in range(len(corpus)):
    rake_precision.append(len(rake_tp[i]) / max(0.01, (len(rake_tp[i]) + len(rake_fp[i]))))
    rake_recall.append(len(rake_tp[i]) / max(0.01, (len(rake_tp[i]) + len(rake_fn[i]))))
    rake_f1.append(2 * (rake_precision[i] * rake_recall[i]) / max(0.01, (rake_precision[i] + rake_recall[i])))
    
print(f'''
Mean RAKE precision: {mean(rake_precision)},
Mean RAKE recall: {mean(rake_recall)},
Mean RAKE F1-score: {mean(rake_f1)}
''')

# total RAKE as if all texts were one 

rake_precision = len([j for i in rake_tp for j in i]) / (len([j for i in rake_tp for j in i]) + len([j for i in rake_fp for j in i]))
rake_recall = len([j for i in rake_tp for j in i]) / (len([j for i in rake_tp for j in i]) + len([j for i in rake_fn for j in i]))
rake_f1 = 2 * (rake_precision * rake_recall) / max(0.01, (rake_precision + rake_recall))
print(f'''
Total RAKE precision: {rake_precision},
Total RAKE recall: {rake_recall},
Total RAKE F1-score: {rake_f1}
''')



Mean RAKE precision: 0.245,
Mean RAKE recall: 0.10511904761904761,
Mean RAKE F1-score: 0.14547119547119547


Total RAKE precision: 0.29411764705882354,
Total RAKE recall: 0.10638297872340426,
Total RAKE F1-score: 0.15625



In [80]:
# mean TextRank across all texts

sum_precision, sum_recall, sum_f1 = [], [], []
for i in range(len(corpus)):
    sum_precision.append(len(sum_tp[i]) / max(0.01, (len(sum_tp[i]) + len(sum_fp[i]))))
    sum_recall.append(len(sum_tp[i]) / max(0.01, (len(sum_tp[i]) + len(sum_fn[i]))))
    sum_f1.append(2 * (sum_precision[i] * sum_recall[i]) / max(0.01, (sum_precision[i] + sum_recall[i])))
    
print(f'''
Mean TextRank precision: {mean(sum_precision)},
Mean TextRank recall: {mean(sum_recall)},
Mean TextRank F1-score: {mean(sum_f1)}
''')

# total TextRank as if all texts were one 

sum_precision = len([j for i in sum_tp for j in i]) / (len([j for i in sum_tp for j in i]) + len([j for i in sum_fp for j in i]))
sum_recall = len([j for i in sum_tp for j in i]) / (len([j for i in sum_tp for j in i]) + len([j for i in sum_fn for j in i]))
sum_f1 = 2 * (sum_precision * sum_recall) / max(0.01, (sum_precision + sum_recall))
print(f'''
Total TextRank precision: {sum_precision},
Total TextRank recall: {sum_recall},
Total TextRank F1-score: {sum_f1}
''')



Mean TextRank precision: 0.05951952494157062,
Mean TextRank recall: 0.29238095238095235,
Mean TextRank F1-score: 0.09742602736615567


Total TextRank precision: 0.06349206349206349,
Total TextRank recall: 0.3404255319148936,
Total TextRank F1-score: 0.10702341137123746



In [81]:
# mean masked TF-IDF across all texts

tfidf_m_precision, tfidf_m_recall, tfidf_m_f1 = [], [], []
for i in range(len(corpus)):
    tfidf_m_precision.append(len(tfidf_m_tp[i]) / (len(tfidf_m_tp[i]) + len(tfidf_m_fp[i])))
    tfidf_m_recall.append(len(tfidf_m_tp[i]) / (len(tfidf_m_tp[i]) + len(tfidf_m_fn[i])))
    tfidf_m_f1.append(2 * (tfidf_m_precision[i] * tfidf_m_recall[i]) / (tfidf_m_precision[i] + tfidf_m_recall[i]))
    
print(f'''
Mean masked TF-IDF precision: {mean(tfidf_m_precision)},
Mean masked TF-IDF recall: {mean(tfidf_m_recall)},
Mean masked TF-IDF F1-score: {mean(tfidf_m_f1)}
''')

# total masked TF-IDF as if all texts were one 

tfidf_m_precision = len([j for i in tfidf_m_tp for j in i]) / (len([j for i in tfidf_m_tp for j in i]) + len([j for i in tfidf_m_fp for j in i]))
tfidf_m_recall = len([j for i in tfidf_m_tp for j in i]) / (len([j for i in tfidf_m_tp for j in i]) + len([j for i in tfidf_m_fn for j in i]))
tfidf_m_f1 = 2 * (tfidf_m_precision * tfidf_m_recall) / (tfidf_m_precision + tfidf_m_recall)
print(f'''
Total masked TF-IDF precision: {tfidf_m_precision},
Total masked TF-IDF recall: {tfidf_m_recall},
Total masked TF-IDF F1-score: {tfidf_m_f1}
''')



Mean masked TF-IDF precision: 0.24666666666666667,
Mean masked TF-IDF recall: 0.8460714285714286,
Mean masked TF-IDF F1-score: 0.3681473198807295


Total masked TF-IDF precision: 0.24666666666666667,
Total masked TF-IDF recall: 0.7872340425531915,
Total masked TF-IDF F1-score: 0.3756345177664975



In [82]:
# mean masked RAKE across all texts

rake_m_precision, rake_m_recall, rake_m_f1 = [], [], []
for i in range(len(corpus)):
    rake_m_precision.append(len(rake_m_tp[i]) / max(0.01, (len(rake_m_tp[i]) + len(rake_m_fp[i]))))
    rake_m_recall.append(len(rake_m_tp[i]) / (len(rake_m_tp[i]) + len(rake_m_fn[i])))
    rake_m_f1.append(2 * (rake_m_precision[i] * rake_m_recall[i]) / max(0.01, (rake_m_precision[i] + rake_m_recall[i])))
    
print(f'''
Mean masked RAKE precision: {mean(rake_m_precision)},
Mean masked RAKE recall: {mean(rake_m_recall)},
Mean masked RAKE F1-score: {mean(rake_m_f1)}
''')

# total masked RAKE as if all texts were one 

rake_m_precision = len([j for i in rake_m_tp for j in i]) / max(0.01, (len([j for i in rake_m_tp for j in i]) + len([j for i in rake_m_fp for j in i])))
rake_m_recall = len([j for i in rake_m_tp for j in i]) / (len([j for i in rake_m_tp for j in i]) + len([j for i in rake_m_fn for j in i]))
rake_m_f1 = 2 * (rake_m_precision * rake_m_recall) / max(0.01, (rake_m_precision + rake_m_recall))
print(f'''
Total masked RAKE precision: {rake_m_precision},
Total masked RAKE recall: {rake_m_recall},
Total masked RAKE F1-score: {rake_m_f1}
''')



Mean masked RAKE precision: 0.18333333333333332,
Mean masked RAKE recall: 0.07261904761904761,
Mean masked RAKE F1-score: 0.1007936507936508


Total masked RAKE precision: 0.2,
Total masked RAKE recall: 0.06382978723404255,
Total masked RAKE F1-score: 0.0967741935483871



In [83]:
# mean masked TextRank across all texts

sum_m_precision, sum_m_recall, sum_m_f1 = [], [], []
for i in range(len(corpus)):
    sum_m_precision.append(len(sum_m_tp[i]) / max(0.01, (len(sum_m_tp[i]) + len(sum_m_fp[i]))))
    sum_m_recall.append(len(sum_m_tp[i]) / (len(sum_m_tp[i]) + len(sum_m_fn[i])))
    sum_m_f1.append(2 * (sum_m_precision[i] * sum_m_recall[i]) / max(0.01, (sum_m_precision[i] + sum_m_recall[i])))
    
print(f'''
Mean masked TextRank precision: {mean(sum_m_precision)},
Mean masked TextRank recall: {mean(sum_m_recall)},
Mean masked TextRank F1-score: {mean(sum_m_f1)}
''')

# total masked TextRank as if all texts were one 

sum_m_precision = len([j for i in sum_m_tp for j in i]) / max(0.01, (len([j for i in sum_m_tp for j in i]) + len([j for i in sum_m_fp for j in i])))
sum_m_recall = len([j for i in sum_m_tp for j in i]) / (len([j for i in sum_m_tp for j in i]) + len([j for i in sum_m_fn for j in i]))
sum_m_f1 = 2 * (sum_m_precision * sum_m_recall) / max(0.01, (sum_m_precision + sum_m_recall))
print(f'''
Total masked TextRank precision: {sum_m_precision},
Total masked TextRank recall: {sum_m_recall},
Total masked TextRank F1-score: {sum_m_f1}
''')



Mean masked TextRank precision: 0.14583135818429935,
Mean masked TextRank recall: 0.5866666666666667,
Mean masked TextRank F1-score: 0.22848567608861725


Total masked TextRank precision: 0.12556053811659193,
Total masked TextRank recall: 0.5957446808510638,
Total masked TextRank F1-score: 0.2074074074074074



### 6. Описать ошибки автоматического выделения ключевых слов (что выделяется лишнее, что не выделяется); предложить свои методы решения этих проблем.

Мне кажется, низкий результат особенно у RAKE и TextRank в первую очередь объясняется длиной текстов. Если TF-IDF берёт внимание весь корпус и получает благодаря этому F1 score 0.5--0.6, то другие два алгоритма учатся только на одном тексте; а самые короткие тексты в этом корпусе длиной 200-300 слов, и алгоритмам недостаточно данных для качественной работы. Иногда они выделяют слишком много ключевых слов, и из-за этого падает precision.  

Можно было бы ограничивать количество выбираемых ключевых слов искуственно, подгоняя его под количество слов в эталоне для этого текста. Это могло бы улучшить результат TextRank, который выделяет много ключевых слов; однако это не получится сделать для RAKE, который даже так в некоторых текстах не смог выделить ключевые слова для порога minFrequency=2. Снижать этот порог кажется неразумным, потому что все ключевые слова (в том числе эталонные) встречаются в тексте не менее 2-х раз. 

По итогу, мне кажется, что TF-IDF показал лучший результат, потому что к такому формату коррпуса он подходит лучше всего.